# HW 1 Classification

In this homework you will be building several varieties of text classifiers.

## Goal

We ask that you construct the following models in PyTorch:

1. A naive Bayes unigram classifer (follow Wang and Manning http://www.aclweb.org/anthology/P/P12/P12-2.pdf#page=118: you should only implement Naive Bayes, not the combined classifer with SVM).
2. A logistic regression model over word types (you can implement this as $y = \sigma(\sum_i W x_i + b)$) 
3. A continuous bag-of-word neural network with embeddings (similar to CBOW in Mikolov et al https://arxiv.org/pdf/1301.3781.pdf).
4. A simple convolutional neural network (any variant of CNN as described in Kim http://aclweb.org/anthology/D/D14/D14-1181.pdf).
5. Your own extensions to these models...

Consult the papers provided for hyperparameters. 


## Setup

This notebook provides a working definition of the setup of the problem itself. You may construct your models inline or use an external setup (preferred) to build your system.

In [4]:
# Text text processing library and methods for pretrained word embeddings
import torch
import numpy as np
import torchtext
from torchtext.vocab import Vectors, GloVe
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

The dataset we will use of this problem is known as the Stanford Sentiment Treebank (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf). It is a variant of a standard sentiment classification task. For simplicity, we will use the most basic form. Classifying a sentence as positive or negative in sentiment. 

To start, `torchtext` requires that we define a mapping from the raw text data to featurized indices. These fields make it easy to map back and forth between readable data and math, which helps for debugging.

In [5]:
# Our input $x$
TEXT = torchtext.data.Field()

# Our labels $y$
LABEL = torchtext.data.Field(sequential=False)

Next we input our data. Here we will use the standard SST train split, and tell it the fields.

In [6]:
train, val, test = torchtext.datasets.SST.splits(
    TEXT, LABEL,
    filter_pred=lambda ex: ex.label != 'neutral')

Let's look at this data. It's still in its original form, we can see that each example consists of a label and the original words.

In [7]:
vars(train[0])

{'label': 'positive',
 'text': ['The',
  'Rock',
  'is',
  'destined',
  'to',
  'be',
  'the',
  '21st',
  'Century',
  "'s",
  'new',
  '``',
  'Conan',
  "''",
  'and',
  'that',
  'he',
  "'s",
  'going',
  'to',
  'make',
  'a',
  'splash',
  'even',
  'greater',
  'than',
  'Arnold',
  'Schwarzenegger',
  ',',
  'Jean-Claud',
  'Van',
  'Damme',
  'or',
  'Steven',
  'Segal',
  '.']}

In [8]:
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

len(train) 6920
vars(train[0]) {'text': ['The', 'Rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'Century', "'s", 'new', '``', 'Conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'Arnold', 'Schwarzenegger', ',', 'Jean-Claud', 'Van', 'Damme', 'or', 'Steven', 'Segal', '.'], 'label': 'positive'}


In order to map this data to features, we need to assign an index to each word an label. The function build vocab allows us to do this and provides useful options that we will need in future assignments.

In [9]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))
print('len(LABEL.vocab)', len(LABEL.vocab))

len(TEXT.vocab) 16284
len(LABEL.vocab) 3


Finally we are ready to create batches of our training data that can be used for training and validating the model. This function produces 3 iterators that will let us go through the train, val and test data. 

In [10]:
train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits((train, val, test), batch_size=10, device=-1, repeat=False)

Let's look at a single batch from one of these iterators. The library automatically converts the underlying words into indices. It then produces tensors for batches of x and y. In this case it will consist of the number of words of the longest sentence (with padding) followed by the number of batches. We can use the vocabulary dictionary to convert back from these indices to words.

In [11]:
batch = next(iter(train_iter))
print("Size of text batch [max sent length, batch size]", batch.text.size())
print("Second in batch", batch.text[:, 0])
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[:, 7].data]))

Size of text batch [max sent length, batch size] torch.Size([20, 10])
Second in batch Variable containing:
  1266
     6
     4
    18
     9
    48
  2207
   286
    11
   237
     4
 14201
  6813
   853
    89
    27
     7
   682
  3447
     2
[torch.LongTensor of size 20]

Converted back to string:  Viewers will need all the luck they can muster just figuring out who 's who in this pretentious mess .


Similarly it produces a vector for each of the labels in the batch. 

In [12]:
print("Size of label batch [batch size]", batch.label.size())
print("Second in batch", batch.label[3])
print("Converted back to string: ", LABEL.vocab.itos[batch.label.data[3]])

Size of label batch [batch size] torch.Size([10])
Second in batch Variable containing:
 2
[torch.LongTensor of size 1]

Converted back to string:  negative


Finally the Vocab object can be used to map pretrained word vectors to the indices in the vocabulary. This will be very useful for part 3 and 4 of the problem.  

In [13]:
# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

print("Word embeddings size ", TEXT.vocab.vectors.size())
print("Word embedding of 'follows', first 10 dim ", TEXT.vocab.vectors[TEXT.vocab.stoi['follows']][:10])

Word embeddings size  torch.Size([16284, 300])
Word embedding of 'follows', first 10 dim  
 0.3925
-0.4770
 0.1754
-0.0845
 0.1396
 0.3722
-0.0878
-0.2398
 0.0367
 0.2800
[torch.FloatTensor of size 10]



## Assignment

Now it is your turn to build the models described at the top of the assignment. 

Using the data given by this iterator, you should construct 4 different torch models that take in batch.text and produce a distribution over labels. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition:  https://www.kaggle.com/c/harvard-cs281-hw1

In [14]:
TEXT.vocab.stoi['but']

19

In [15]:
TEXT.vocab.stoi['despite']

575

In [16]:
TEXT.vocab.stoi['unless']

5026

In [17]:
TEXT.vocab.stoi['while']

195

In [18]:
TEXT.vocab.stoi['however']

1014

In [19]:
def semantic_transform(batch):
    batch_trans = torch.t(batch.text)
    batch_t = batch_trans.clone()

    for i in range(len(batch_t)):
        but = np.where(batch_t[i].data.numpy() == 19)[0]
        despite = np.where(batch_t[i].data.numpy() == 575)[0]
        unless = np.where(batch_t[i].data.numpy() == 5026)[0]
        #whil = np.where(batch_t[i].data.numpy() == 195)[0]
        however = np.where(batch_t[i].data.numpy() == 1014)[0]
        if len(but) > 0:
            batch_t[i] = torch.cat((batch_t[i][but[0] + 1:] , torch.LongTensor(np.ones(but[0]+1).astype(int))))
        if len(despite) > 0 and despite[0] > 3:
            batch_t[i] = torch.cat((batch_t[i][:despite[0]] , torch.LongTensor(np.ones(batch_t.size()[1] - despite[0]).astype(int))))
        if len(unless) > 0 and unless[0] > 3:
            batch_t[i] = torch.cat((batch_t[i][:unless[0]] , torch.LongTensor(np.ones(batch_t.size()[1] - unless[0]).astype(int))))
        if len(however) > 0:
            batch_t[i] = torch.cat((batch_t[i][however[0] + 1:] , torch.LongTensor(np.ones(however[0]+1).astype(int))))
    return torch.t(batch_t)

In [43]:
# Our input $x$
TEXT = torchtext.data.Field()
# Our labels $y$
LABEL = torchtext.data.Field(sequential=False)
train, val, test = torchtext.datasets.SST.splits(
    TEXT, LABEL,
    filter_pred=lambda ex: ex.label != 'neutral')
TEXT.build_vocab(train)
LABEL.build_vocab(train)
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

filter_sizes = [3, 4, 5]
filter_num = 50
dropout_rate = 0.5
sent_length = max([len(vars(train[i])['text']) for i in range(len(train))] 
                   +[ len(vars(val[i])['text']) for i in range(len(val))] 
                  +[ len(vars(test[i])['text']) for i in range(len(test))]) + 30
class LCA_CNN(nn.Module):
    def __init__(self):
        super(LCA_CNN, self).__init__()
        self.embed = nn.Embedding(16284, 300)
        self.embed.weight.data = TEXT.vocab.vectors
        components = []
        for size in filter_sizes:
            components.append(
            nn.Sequential(
                nn.Conv1d(1, filter_num, size * 300, stride=300) ,
                nn.MaxPool1d(kernel_size = sent_length - size + 1 + 30),
            nn.Tanh())   
            )
        self.layers = nn.ModuleList(components)
        self.fc = nn.Linear(filter_num * len(filter_sizes), 2)
        self.training = True
    def forward(self, text):
        x = self.embed(torch.t(text))
        x = x.view(x.size(0), 1, -1)
        x_list= [convolve(x) for convolve in self.layers]
        out = torch.cat(x_list, 2)
        out = out.view(out.size(0), -1)
        out = F.dropout(out, p=dropout_rate, training=self.training)
        return F.log_softmax(self.fc(out)), out
    
    def train(self):
        train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train, val, test), batch_size=10, device=-1, repeat=False)
        loss_fn = nn.NLLLoss()
        optimizer = optim.Adam(self.parameters())
        for epoch in range(10):
            tic = time.time()
            for batch in train_iter:
                self.training=True
                self.zero_grad()   
                
                batch.text = semantic_transform(batch)
                
                if sent_length > batch.text.size()[0]:
#                     padded_batch = torch.cat((batch.text ,
#                         torch.LongTensor(np.ones((sent_length-batch.text.size()[0], batch.text.size()[1])).astype(int))))
                    padded_batch = torch.cat((batch.text ,
    torch.LongTensor(np.ones((sent_length-batch.text.size()[0], batch.text.size()[1])).astype(int))))
                    padded_batch = torch.cat((torch.LongTensor(np.ones((30, batch.text.size()[1])).astype(int)), padded_batch.data))
                else:
                    padded_batch = batch
                log_probs, _ = self.forward(padded_batch)
                loss = loss_fn(log_probs, batch.label - 1)
                loss.backward()
                optimizer.step()
#                 if self.fc.weight.norm().data[0] > 3:
#                     self.fc.weight.data = self.fc.weight.data * 3 / self.fc.weight.data.norm()
            predictions = []
            real_labels = []
            for batch in val_iter:
                self.training=False  
                
                batch.text = semantic_transform(batch)
                
                if sent_length > batch.text.size()[0]:
#                     padded_batch = torch.cat((batch.text ,
#                         torch.LongTensor(np.ones((sent_length-batch.text.size()[0], batch.text.size()[1])).astype(int))))
                    padded_batch = torch.cat((batch.text ,
    torch.LongTensor(np.ones((sent_length-batch.text.size()[0], batch.text.size()[1])).astype(int))))
                    padded_batch = torch.cat((torch.LongTensor(np.ones((30, batch.text.size()[1])).astype(int)), padded_batch.data))                
                else:
                    padded_batch = batch
                log_probs, _ = self.forward(padded_batch)
                loss = loss_fn(log_probs, batch.label - 1)
#                 if self.fc.weight.norm().data[0] > 3:
#                     self.fc.weight.data = self.fc.weight.data * 3 / self.fc.weight.data.norm()
                preds = torch.max(log_probs, 1)[1]
                predictions += list(preds.data.numpy())
                real_labels += list(batch.label.data.numpy() - 1)                
            eval_acc = 1.0 * np.sum(np.array(predictions) == np.array(real_labels))/ len(predictions)
            print('[epoch: {:d}] train_loss: {:.3f}   acc: {:.3f}   ({:.1f}s)'.format(epoch, loss.data[0], eval_acc, time.time()-tic) )


In [44]:
model_2 = LCA_CNN()

In [45]:
model_2.train()

[epoch: 0] train_loss: 0.750   acc: 0.786   (97.9s)
[epoch: 1] train_loss: 0.607   acc: 0.791   (127.8s)
[epoch: 2] train_loss: 0.145   acc: 0.798   (130.9s)


KeyboardInterrupt: 

In [21]:
def to_1hot(sentence):
    _1hot = torch.zeros(len(TEXT.vocab))
    for word in sentence.data:
        _1hot[word] = 1
        
    return _1hot

In [33]:
class NB_uni():
    def __init__(self, vocab_size):
        self.p = nn.Embedding(vocab_size,1)
        self.q = nn.Embedding(vocab_size,1)
        self.p.weight.data = torch.Tensor(np.ones(vocab_size))
        self.q.weight.data = torch.Tensor(np.ones(vocab_size))
        self.N_plus = 0
        self.N_minus = 0

    def forward(self, text):
        w = torch.log((self.p.weight / torch.sum(torch.abs(self.p.weight))) /  (self.q.weight / torch.sum(torch.abs(self.q.weight)))).data
        b = np.log(self.N_plus / self.N_minus)
        output = []
        sentences = torch.t(text)
        for sent in sentences:
            x = torch.from_numpy(np.unique(sent.data.numpy()))
            summed = torch.sum(w[x], dim=0)
            output.append(int(torch.sign(summed + b)[0]))
        return output
    
    def predict(self, text):
        predicts = []
        text = Variable(torch.LongTensor(text.data.numpy().clip(min=0)))
        forward = self.forward(text)
        for i in forward:
            if i == -1:
                predicts.append(2)
            else:
                predicts.append(1)
        return predicts
                
    
    def train(self):
        for batch in train_iter:
            #batch.text = Variable(torch.LongTensor(batch.text.data.numpy().clip(min=0)))
            batch.text = semantic_transform(batch)
            self.N_plus += torch.sum(batch.label == 1).data[0]
            self.N_minus += torch.sum(batch.label == 2).data[0]
            if torch.sum(batch.label == 1).data[0] > 0:
                pos = torch.t(batch.text)[torch.squeeze(torch.nonzero((batch.label == 1).data))]
                self.p.weight.data += np.sum(np.apply_along_axis(to_1hot, 1, pos.data.numpy()))
            if torch.sum(batch.label == 2).data[0] > 0:
                neg = torch.t(batch.text)[torch.squeeze(torch.nonzero((batch.label == 2).data))]
                self.q.weight.data += np.sum(np.apply_along_axis(to_1hot, 1, neg.data.numpy()))

In [34]:
def test(model):
    "All models should be able to be run with following command."
    upload = []
    # Update: for kaggle the bucket iterator needs to have batch_size 10
    #test_iter = torchtext.data.BucketIterator(test, train=False, batch_size=10)
    for batch in test_iter:
        # Your prediction data here (don't cheat!)
        probs = model.predict(batch.text)
        upload += probs
    return upload

In [35]:
model_1 = NB_uni(TEXT.vocab.vectors.size()[0])
model_1.train()
upload = []
for batch in test_iter:
    # Your prediction data here (don't cheat!)
    batch.text = semantic_transform(batch)
    probs = model_1.predict(batch.text)
    upload += probs
#         _, argmax = probs.max(1)
#         upload += list(argmax.data)

In [36]:
test_labels = []
for batch in test_iter:
    test_labels += list(batch.label.data.numpy())

In [37]:
(len(upload) - np.sum(np.abs(np.array(upload) - np.array(test_labels)))) / len(upload)

0.82976386600768803

In [651]:
def test_model(model):
    "All models should be able to be run with following command."
    upload = []
    # Update: for kaggle the bucket iterator needs to have batch_size 10
    #test_iter = torchtext.data.BucketIterator(test, train=False, batch_size=10)
    for batch in test_iter:
        # Your prediction data here (don't cheat!)
        probs = model.predict(batch.text)
        upload += probs
#         _, argmax = probs.max(1)
#         upload += list(argmax.data)
    print(len(upload))
    with open("predictions.txt", "w") as f:
        f.write('Id,Cat' + '\n')
        for u in range(len(upload)):
            f.write(str(u) + ',' + str(upload[u]) + "\n")

In [652]:
test_model(model_1)

1821


In addition, you should put up a (short) write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/